In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import sklearn

import openpyxl
import xlwt

## Case # 1
### Discharge curve &#8594; Capacity rate curve
### E vs Li (<font color=red>V</font>) vs Capacity (<font color=red>Ah cm<sup>-2</sup></font>) &#8594; Capacity (<font color=red>Ah cm<sup>-2</sup></font>) vs i (<font color=red>$\mu A$ cm <sup>-2</sup></font>)

### Step 1) Upload and convert potential vs capacity data

In [2]:
def potential_rate(xls_file, sheet_name, paper_num, set_num, c_rate):
    """ 
    This function converts potential vs. capacity data to capacity vs c-rate data
    The Highest x-value (capacity [mAh]) from the charge/discharge graph
    PARAMETERS
    ----------
    1) Excel file (file path) - string
    2) sheetnames - list
    3) Paper # - string
    4) Set # - integer: max number of sets
    5) c_rate - dataframe
    RETURNS
    -------
    Capacity vs c-rate - dataframe
    """
    # Determining which set # and the number of set lists from the excel file
    set_list = ['set #' + str(i) for i in range (1, set_num + 1)]
    # Test that the input 'set_num' is an integer
    assert type(set_num) == int, 'set_num must be an integer'
    # Dataframing the interested potential vs capacity excel sheet
    df = pd.read_excel(xls_file, sheet_name, header = [0,1,2]) #
    # Merging multiple spreadsheets
    df_sheets = []
    for i in sheet_name:
        df_sheets.append(df[i])
    df_merged = pd.concat(df_sheets, axis = 1)
    # Selecting maximum capacity values for each dataset and concatnate with corresponding c-rates or current density
    caplist = []
    for i in set_list:
        set_i = (df_merged[paper_num, i])
        set_i_max = set_i["Capacity (mAh/g)"].max(axis=0).array
        caplist.append(c_rate)
        caplist.append(pd.DataFrame({"Capacity (mAh/g)": set_i_max}))
    df_cap_rate = pd.concat(caplist, axis = 1)
    # Test that the output is a dataframe
    assert type(c_rate) == type(df_cap_rate), 'The output must be a dataframe'
    return df_cap_rate

In [3]:
file_path = '~/capstone_spring_2021/3DLi-ionbattery/liibattery3d/data/liibattery3d_performancelog.xls'
sheet_name = ['1_Discharge (Cover2)', '1_Discharge (1C)', '1_Discharge (2C)', '1_Discharge (5C)']
rates = { "C rate": [0.5, 1.0, 2.0, 5.0]}
c_rate = pd.DataFrame(rates)
df_cap_rate = potential_rate(file_path, sheet_name, 'Paper # 32', 3, c_rate)
df_cap_rate

/home/gskl92/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2940: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


,C rate,Capacity (mAh/g),C rate,Capacity (mAh/g),C rate,Capacity (mAh/g)
0,0.5,154.154324,0.5,147.303752,0.5,144.812819
1,1.0,149.841197,1.0,139.399821,1.0,135.372662
2,2.0,141.685172,2.0,118.804436,2.0,116.264876
3,5.0,110.398826,5.0,65.780351,5.0,48.477729


### Step 2) Export the new dataframe to an existing excel file

In [4]:
def excel_merge(dataframe, xls_file, sheetname):
    """
    This function adds the converted dataframe to an existing excel file
    PARAMETERS
    ----------
    1) converted dataframe - dataframe
    2) Excel file (file path) - string
    3) sheetname - string
    RETURNS
    -------
    A new sheet of the convereted data in the excel file
    """
    # Exporting the converted dataframe to an excel file
    dataframe.to_excel(xls_file, sheet_name = sheetname, index=False, header=True)
    # Test that the sheet name is a string
    assert type(sheetname) == str, 'sheetname must be a string'
    print('saved succesfully to' + xls_file)
    return

In [5]:
file_path_temp = '~/capstone_spring_2021/3DLi-ionbattery/Demo/Excelfile_ex.xls'
excel_merge(df_cap_rate, file_path_temp, 'Capacity_Rate')

saved succesfully to~/capstone_spring_2021/3DLi-ionbattery/Demo/Excelfile_ex.xls


<ipython-input-4-361c402f5620>:14: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  dataframe.to_excel(xls_file, sheet_name = sheetname, index=False, header=True)


## Case # 2
### Rate test data &#8594; Capacity rate curve
### Capacity (<font color=red>Ah cm<sup>-2</sup></font>) vs Cycle #  &#8594; Capacity (<font color=red>Ah cm<sup>-2</sup></font>) vs i (<font color=red>$\mu A$ cm <sup>-2</sup></font>)